In [9]:
using Pkg
Pkg.add("Distributions")
Pkg.add("ROCAnalysis")
Pkg.build("MLBase")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [10]:
# Authors: Hamza Tazi Bouardi & Pierre-Henri Ramirez
using JuMP, Gurobi, CSV, LinearAlgebra, DataFrames, Random, Distributions, Statistics,MLBase, ROCAnalysis
gurobi_env = Gurobi.Env()

### Loading Data ###
# train_data = CSV.read("data/adult_train.csv")
# X_train = convert(Matrix, train_data[:, 1:91])
# y_train = train_data[:, 92]
# test_data = CSV.read("data/adult_test.csv")
# X_test = convert(Matrix, test_data[:, 1:91])
# y_test = test_data[:, 92]
# println("Got the data for X dataset.")



┌ Info: Precompiling MLBase [f0e99cf1-93fa-52ec-9ecc-5026115318e0]
└ @ Base loading.jl:1192
┌ Info: Precompiling ROCAnalysis [f535d66d-59bb-5153-8d2b-ef0a426c6aff]
└ @ Base loading.jl:1192



--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007fddd055da00)

In [4]:
function one_hot_encode(X, names)
    X2 = deepcopy(X)
    deletecols!(X2, Symbol.(names))
    for i in names
        vales = unique(X[i])
        for j in vales
           X2[Symbol(string(i)*"_"*string(j))] = (X[i].==j)*1
        end
    end
    return X2
end

one_hot_encode (generic function with 1 method)

In [5]:
caesarian_df = CSV.read("caesarian.csv";header=true)
categorical_vars = Symbol.(["Delivery number" ;"Delivery time";"Blood of Pressure";"Heart Problem"])
X = one_hot_encode(caesarian_df[:,1:end-1], categorical_vars) 
y = caesarian_df[:,end]
;

┌ Warning: `deletecols!(df::DataFrame, inds)` is deprecated, use `select!(df, Not(inds))` instead.
│   caller = one_hot_encode(::DataFrame, ::Array{Symbol,1}) at In[4]:3
└ @ Main ./In[4]:3
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = one_hot_encode(::DataFrame, ::Array{Symbol,1}) at In[4]:5
└ @ Main ./In[4]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = one_hot_encode(::DataFrame, ::Array{Symbol,1}) at In[4]:7
└ @ Main ./In[4]:7
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = one_hot_encode(::DataFrame, ::Array{Symbol,1}) at In[4]:7
└ @ Main ./In[4]:7


In [ ]:
function solve_problem(w_k, y, X, K, λ)
    n, p = size(X)
    model_inner_max = Model(solver=GurobiSolver(OutputFlag=0,gurobi_env))
    @variable(model_inner_max, z[1:n] >= 0)
    @constraint(model_inner_max, [i=1:n], 1 >= z[i])
    @constraint(model_inner_max, sum(z) <= K)
    @objective(
        model_inner_max,
        Max,
        sum(z[i]*log(1+exp(-y[i]*dot(X[i,:], w_k))) for i=1:n)
    )
    solve(model_inner_max)
    optimal_z_k = getvalue(z)
    optimal_f_k = getobjectivevalue(model_inner_max) + λ*dot(w_k,w_k)
    return optimal_z_k, optimal_f_k
end

In [54]:
function scores(preds, gt)
    acc = sum(preds .== gt)/size(preds)[1]
    TPR = dot((preds.==1),gt.==1)/(dot((preds.==1),gt.==1) + dot((preds.==-1),gt.==1))
    FPR = dot((preds.==1),gt.==-1)/ (dot((preds.==1),gt.==-1) + dot((preds.==-1),gt.==-1))
    return acc, TPR, FPR
end

scores (generic function with 1 method)

In [6]:
### Utils Functions ###
function compute_∇f(w_k, y, X, λ)
    n, p = size(X)
    temp = zeros(p)
    for i in 1:n
        temp = temp + 
        (1+exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k))^(-1)*
        exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
    end
    ∇f_k = temp
    return ∇f_k
end

compute_∇f (generic function with 1 method)

In [170]:
### Cutting Planes Implementation ###
function LR_cutting_planes(y, X, ε, λ)
    errors = []
    n, p = size(X)
    # Initialization values and step 0
    w_0 = [0 for i in 1:p]
    #w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
    f_0 = sum(log(1+exp(-y[i]*dot(X[i,:], w_0)+λ*transpose(w_0)*w_0)) for i=1:n)
    ∇f_0 = compute_∇f(w_0, y, X, λ)

    # Outer minimization problem
    outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    @variable(outer_min_model, t >= 0)
    @variable(outer_min_model, w[1:p])
    #@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
    @constraint(outer_min_model, t >= f_0 + dot(∇f_0, w)-dot(∇f_0, w_0))
    @constraint(outer_min_model, [j=1:p], 1 >= w[j])
    @constraint(outer_min_model, [j=1:p], w[j] >= -1)
    @objective(outer_min_model, Min, t)
    k = 1 # Number of constraints in the final problem
    solve(outer_min_model)

    # New steps k
    t_k = getvalue(t)
    w_k = getvalue(w)
    
    f_k = sum(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)) for i=1:n)


    ∇f_k = compute_∇f(w_k, y, X, λ)
    while abs(f_k - t_k) >= ε # error
        push!(errors, f_k - t_k)
        @constraint(outer_min_model,t >= f_k + dot(∇f_k, w)-dot(∇f_k, w_k))
        k += 1
        solve(outer_min_model)
        # Updating all the values
        t_k = getvalue(t)
        w_k = getvalue(w)
        f_k = sum(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)) for i=1:n)

        ∇f_k = compute_∇f(w_k, y, X, λ)
        if k%100 == 0
            println("Number of constraints: ", k, "\t Error = ", abs(t_k - f_k))
        end
        if k > 20000
            break
        end
    end
    push!(errors, f_k - t_k)
    return t_k, f_k, w_k, errors
end


LR_cutting_planes (generic function with 1 method)

In [119]:
transpose(Array(caesarian_df[1,1:end-1]))*w

8.330554661667191

In [198]:
n, p = size(X)
λ = 0.01
K=100
# Initialization values and step 0
w_0 = [0 for i in 1:p]
#w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
z_0, f_0 = solve_inner_max_problem_o(w_0, y, X, K, λ)
∇f_0 = compute_∇f_o(w_0, z_0, y, X, λ)
println(∇f_0)
outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
@variable(outer_min_model, t >= 0)
@variable(outer_min_model, w[1:p])
#@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
@constraint(outer_min_model, t >= f_0 + dot(∇f_0, w)-dot(∇f_0, w_0))
@constraint(outer_min_model, [j=1:p], 1 >= w[j])
@constraint(outer_min_model, [j=1:p], w[j] >= -1)
@objective(outer_min_model, Min, t)
k = 1 # Number of constraints in the final problem
solve(outer_min_model)

# New steps k
t_k = getvalue(t)
w_k = getvalue(w)
z_k, f_k = solve_inner_max_problem_o(w_k, y, X, K, λ)
∇f_k = compute_∇f_o(w_k, z_k, y, X, λ)
println(∇f_k)

[-644.5, -11.0, -7.5, -3.5, -1.0, -15.0, -4.0, -4.0, -7.0, -8.5, -7.5, -11.0, -12.0]
[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]


In [199]:
w_k

13-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [191]:
n,p = size(X)
λ = 0.01
w_0 = [0 for i in 1:p]
#w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
f_0 = sum(log(1+exp(-y[i]*dot(X[i,:], w_0)+λ*transpose(w_0)*w_0)) for i=1:n)
∇f_0 = compute_∇f(w_0, y, X, λ)
println(∇f_0)
 # Outer minimization problem
outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
@variable(outer_min_model, t >= 0)
@variable(outer_min_model, w[1:p])
#@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
@constraint(outer_min_model, t >= f_0 + dot(∇f_0, w)-dot(∇f_0, w_0))
@constraint(outer_min_model, [j=1:p], 1 >= w[j])
@constraint(outer_min_model, [j=1:p], w[j] >= -1)
@objective(outer_min_model, Min, t)
k = 1 # Number of constraints in the final problem
solve(outer_min_model)

# New steps k
t_k = getvalue(t)
w_k = getvalue(w)

f_k = sum(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)) for i=1:n)

∇f_k = compute_∇f(w_k, y, X, λ)
println(∇f_k)

[-644.5, -11.0, -7.5, -3.5, -1.0, -15.0, -4.0, -4.0, -7.0, -8.5, -7.5, -11.0, -12.0]
[0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069, 0.362069]


In [197]:
f_k

13.992943611198907

In [171]:
t,f,w,e = LR_cutting_planes(caesarian_df[:,end], caesarian_df[:,1:end-1], 0.001, 0.01)

(23.598127664199165, 23.598963220337744, [0.384704, 0.00696594, 0.0433285, 0.0426026, -0.0205498], Any[24.4276, 19.329, 7.66883, 3.86515, 2.85148, 3.88155, 5.64845, 2.56713, 1.80432, 1.56094  …  0.00213084, 0.00327753, 0.00234605, 0.00232466, 0.00123772, 0.00279607, 0.00115183, 0.00201565, 0.00101017, 0.000835556])

In [172]:
w

5-element Array{Float64,1}:
  0.38470391929740677 
  0.006965942108277889
  0.04332852030801294 
  0.04260259231085005 
 -0.020549818328637365

In [173]:
1 ./ (1 .+ exp.(-(Matrix(caesarian_df[:,1:end-1])*w)))

80-element Array{Float64,1}:
 0.9998075837927463
 0.9999571960077634
 0.9999590109618189
 0.9999808632503093
 0.9998006044697094
 0.9999569280867059
 0.9999708649730688
 0.9999957730508368
 0.9999801689972236
 0.999971905338189 
 0.9999990800169115
 0.9999970846676023
 0.9998691184822467
 ⋮                 
 0.9999339190470488
 0.9999721206036951
 0.99856754575615  
 0.9999867233021513
 0.9999801689972236
 0.999995685289523 
 0.9999996229769623
 0.9999721003584779
 0.9999971428860605
 0.9999876133544255
 0.999939403993294 
 0.999915280969719 

In [121]:
for i in 1:80
    println(1/(1+exp(-transpose(Array(X[i,:]))*w)))
end
;

DimensionMismatch: DimensionMismatch("")

In [39]:
print(1/(1+exp(transpose(Array(X[7,:]))*w)))

0.0004995637823444937

In [76]:
w_0 = [0 for i in 1:13]
transpose(w_0)*Array(X[1,:])

0

In [ ]:
temp + (1+exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k))^(-1).*
        (exp.(-y[i]*dot(w_k,X[i,:]))*(-y[i]*X[i,:]) .+ 2*λ*w_k)

In [202]:
function compute_∇f_o(w_k, z_k, y, X, λ)
    n, p = size(X)
    ∇f_k = sum(-z_k[i]/(1+exp(y[i]*dot(w_k,X[i,:])))*y[i]*Array(X[i,:]) for i in 1:n) .+ 2*λ*w_k
#     temp = zeros(p)
#     for i in 1:n
#         temp = temp + 
#         (1+exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k))^(-1)*
#         exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
#     end
#     ∇f_k = temp
    return ∇f_k
end

function solve_inner_max_problem_o(w_k, y, X, K, λ)
    n, p = size(X)
    model_inner_max = Model(solver=GurobiSolver(OutputFlag=0,gurobi_env))
    @variable(model_inner_max, z[1:n] >= 0)
    @constraint(model_inner_max, [i=1:n], 1 >= z[i])
    @constraint(model_inner_max, sum(z) <= K)
    @objective(
        model_inner_max,
        Max,
        sum(z[i]*log(1+exp(-y[i]*dot(X[i,:], w_k))) for i=1:n)
    )
    solve(model_inner_max)
    optimal_z_k = getvalue(z)
    optimal_f_k = getobjectivevalue(model_inner_max) + λ*dot(w_k,w_k)
    return optimal_z_k, optimal_f_k
end

### Cutting Planes Implementation ###
function stable_LR_cutting_planes_o(y, X, ε, K,λ)
    errors = []
    n, p = size(X)
    # Initialization values and step 0
    w_0 = [0 for i in 1:p]
    #w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
    z_0, f_0 = solve_inner_max_problem_o(w_0, y, X, K, λ)
    ∇f_0 = compute_∇f_o(w_0, z_0, y, X, λ)

    # Outer minimization problem
    outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    @variable(outer_min_model, t >= 0)
    @variable(outer_min_model, w[1:p])
    #@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
    @constraint(outer_min_model, t >= f_0 + dot(∇f_0, w)-dot(∇f_0, w_0))
    @constraint(outer_min_model, [j=1:p], 1 >= w[j])
    @constraint(outer_min_model, [j=1:p], w[j] >= -1)
    @objective(outer_min_model, Min, t)
    k = 1 # Number of constraints in the final problem
    solve(outer_min_model)

    # New steps k
    t_k = getvalue(t)
    w_k = getvalue(w)
    z_k, f_k = solve_inner_max_problem_o(w_k, y, X, K, λ)

    ∇f_k = compute_∇f_o(w_k, z_k, y, X, λ)
    while abs(f_k - t_k) >= ε # error
        push!(errors, f_k - t_k)
        @constraint(outer_min_model,t >= f_k + dot(∇f_k, w)-dot(∇f_k, w_k))
        k += 1
        solve(outer_min_model)
        # Updating all the values
        t_k = getvalue(t)
        w_k = getvalue(w)
        z_k, f_k = solve_inner_max_problem_o(w_k, y, X, K, λ)

        ∇f_k = compute_∇f_o(w_k, z_k, y, X, λ)
        if k%100 == 0
            println("Number of constraints: ", k, "\t Error = ", abs(t_k - f_k))
        end
        if k > 20000
            break
        end
    end
    push!(errors, f_k - t_k)
    return t_k, f_k, w_k, z_k, errors
end


stable_LR_cutting_planes_o (generic function with 1 method)

In [ ]:
t,f,w,e = LR_cutting_planes(caesarian_df[:,end], caesarian_df[:,1:end-1], 0.001, 0.01)

In [179]:
t,f,w,e = LR_cutting_planes(caesarian_df[:,end], caesarian_df[:,1:end-1], 0.001, 0.01)

(23.598127664199165, 23.598963220337744, [0.384704, 0.00696594, 0.0433285, 0.0426026, -0.0205498], Any[24.4276, 19.329, 7.66883, 3.86515, 2.85148, 3.88155, 5.64845, 2.56713, 1.80432, 1.56094  …  0.00213084, 0.00327753, 0.00234605, 0.00232466, 0.00123772, 0.00279607, 0.00115183, 0.00201565, 0.00101017, 0.000835556])

In [180]:
1 ./ (1 .+ exp.(-(Matrix(caesarian_df[:,1:end-1])*w)))

80-element Array{Float64,1}:
 0.9998075837927463
 0.9999571960077634
 0.9999590109618189
 0.9999808632503093
 0.9998006044697094
 0.9999569280867059
 0.9999708649730688
 0.9999957730508368
 0.9999801689972236
 0.999971905338189 
 0.9999990800169115
 0.9999970846676023
 0.9998691184822467
 ⋮                 
 0.9999339190470488
 0.9999721206036951
 0.99856754575615  
 0.9999867233021513
 0.9999801689972236
 0.999995685289523 
 0.9999996229769623
 0.9999721003584779
 0.9999971428860605
 0.9999876133544255
 0.999939403993294 
 0.999915280969719 

In [205]:
t,f,w,e = stable_LR_cutting_planes_o(y, X, 0.001, 100, 0.01)

Number of constraints: 100	 Error = 0.011935158525627543


(23.56966796926474, 23.57058493408266, [0.482407, 0.0463461, -0.065342, -0.118804, -0.0427386, -0.00126255, -0.00387642, -0.0553371, 0.00212159, -0.0675339, 0.0624503, 0.0273817, 0.0119962], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0], Any[23.697, 14.8689, 5.48829, 2.14173, 0.939819, 0.542255, 0.480033, 0.420589, 0.393126, 0.310426  …  0.00132884, 0.00193032, 0.00136474, 0.00142469, 0.00171808, 0.00108831, 0.00100686, 0.00178033, 0.00142019, 0.000916965])

In [203]:
t,f,w,e = stable_LR_cutting_planes_o(caesarian_df[:,end], caesarian_df[:,1:end-1], 0.001, 100, 0.01)

(23.56967935467481, 23.57059308727941, [0.481039, 0.0919781, 0.0586182, -0.121395, 0.0730615], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0], Any[23.617, 17.1508, 6.04423, 2.28995, 0.896843, 0.380658, 0.200893, 0.160806, 0.154469, 0.199646  …  0.00470553, 0.00315299, 0.00594476, 0.0021823, 0.00296698, 0.00170896, 0.00192702, 0.00102675, 0.00213877, 0.000913733])

In [204]:
1 ./ (1 .+ exp.(-(Matrix(caesarian_df[:,1:end-1])*w)))

80-element Array{Float64,1}:
 0.9999705278315583
 0.9999965237827755
 0.9999967216943401
 0.9999983558178651
 0.9999761906479103
 0.9999968167282722
 0.9999978512043882
 0.9999998231160252
 0.9999986717400143
 0.9999977783117039
 0.9999999689593898
 0.999999890228798 
 0.999984783072052 
 ⋮                 
 0.999994539371293 
 0.9999983073824691
 0.9997438863189421
 0.9999993409021081
 0.9999986717400143
 0.9999998355786222
 0.9999999907893481
 0.9999979735419771
 0.9999999002689144
 0.9999991257518246
 0.9999951434505961
 0.999991908769859 

In [141]:
w

5-element Array{Float64,1}:
  0.02122501010819794
 -0.1593256752360007 
  0.04141888602590581
 -0.05725501452607942
  0.11747785282823284

In [145]:
w

5-element Array{Float64,1}:
  0.38245186474397164 
  0.04800180858263814 
  0.06281723360773737 
  0.02924388913386663 
 -0.013920802906773465